In [51]:
import pandas as pd
import numpy as np

from flask import Flask, jsonify, render_template
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from pandas_geojson import to_geojson

In [52]:
engine = create_engine("sqlite:///sqlite3/melbournebusinessdatabase.db")
database_df = pd.read_sql("Select * from melbourne_business where census_year = 2020",engine)
# database_df = database_df.iloc[: , 1:]
# database_df['number_of_seats'] = database_df['number_of_seats'].astype('int')
database_df


,Unnamed: 0,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,50816,2020,Melbourne (CBD),551 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Indoor,60,144.9565145,-37.82097941
1,50817,2020,Melbourne (CBD),551 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Outdoor,6,144.9565145,-37.82097941
2,50818,2020,Melbourne (CBD),553 Flinders Street MELBOURNE VIC 3000,Takeaway Food Services,Seats - Indoor,12,144.9565145,-37.82097941
3,50819,2020,Melbourne (CBD),547 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Indoor,44,144.9565145,-37.82097941
4,50820,2020,Melbourne (CBD),563 Flinders Street MELBOURNE VIC 3000,Takeaway Food Services,Seats - Indoor,12,144.9559094,-37.82108687
...,...,...,...,...,...,...,...,...,...
3231,54047,2020,Kensington,61 Stubbs Street KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Outdoor,8,144.9356959,-37.79223169
3232,54048,2020,Kensington,61 Stubbs Street KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Indoor,24,144.9356959,-37.79223169
3233,54049,2020,Kensington,169 Rankins Road KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Indoor,12,144.9311131,-37.79436219
3234,54050,2020,Kensington,173 Rankins Road KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Indoor,12,144.9311131,-37.79436219


In [53]:
not_top10 = database_df.groupby('clue_small_area').count().sort_values('trading_name',ascending = False).index[10:]
not_top10

Index(['Port Melbourne', 'South Yarra', 'West Melbourne (Industrial)'], dtype='object', name='clue_small_area')

In [54]:
database_df = database_df.replace(not_top10, 'Other')

In [55]:
database_df


,Unnamed: 0,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,50816,2020,Melbourne (CBD),551 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Indoor,60,144.9565145,-37.82097941
1,50817,2020,Melbourne (CBD),551 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Outdoor,6,144.9565145,-37.82097941
2,50818,2020,Melbourne (CBD),553 Flinders Street MELBOURNE VIC 3000,Takeaway Food Services,Seats - Indoor,12,144.9565145,-37.82097941
3,50819,2020,Melbourne (CBD),547 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Indoor,44,144.9565145,-37.82097941
4,50820,2020,Melbourne (CBD),563 Flinders Street MELBOURNE VIC 3000,Takeaway Food Services,Seats - Indoor,12,144.9559094,-37.82108687
...,...,...,...,...,...,...,...,...,...
3231,54047,2020,Kensington,61 Stubbs Street KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Outdoor,8,144.9356959,-37.79223169
3232,54048,2020,Kensington,61 Stubbs Street KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Indoor,24,144.9356959,-37.79223169
3233,54049,2020,Kensington,169 Rankins Road KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Indoor,12,144.9311131,-37.79436219
3234,54050,2020,Kensington,173 Rankins Road KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Indoor,12,144.9311131,-37.79436219


In [56]:
database_df.drop_duplicates(subset=['trading_name'])

,Unnamed: 0,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,50816,2020,Melbourne (CBD),551 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Indoor,60,144.9565145,-37.82097941
2,50818,2020,Melbourne (CBD),553 Flinders Street MELBOURNE VIC 3000,Takeaway Food Services,Seats - Indoor,12,144.9565145,-37.82097941
3,50819,2020,Melbourne (CBD),547 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Indoor,44,144.9565145,-37.82097941
4,50820,2020,Melbourne (CBD),563 Flinders Street MELBOURNE VIC 3000,Takeaway Food Services,Seats - Indoor,12,144.9559094,-37.82108687
6,50822,2020,Melbourne (CBD),Sea Life Melbourne Aquarium King Street MELBOU...,Zoological and Botanical Gardens Operation,Seats - Indoor,164,144.9585078,-37.82058853
...,...,...,...,...,...,...,...,...,...
3228,54044,2020,Kensington,36 Stubbs Street KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Indoor,24,144.9352778,-37.79000411
3229,54045,2020,Kensington,72 Stubbs Street KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Outdoor,10,144.9353015,-37.79142168
3231,54047,2020,Kensington,61 Stubbs Street KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Outdoor,8,144.9356959,-37.79223169
3233,54049,2020,Kensington,169 Rankins Road KENSINGTON VIC 3031,Cafes and Restaurants,Seats - Indoor,12,144.9311131,-37.79436219


In [59]:
gpB_df = database_df.groupby('clue_small_area', as_index=False).count()
gpB_df =gpB_df [["clue_small_area", "trading_name"]].sort_values('trading_name',ascending = False)
# gpA_df.sort_values('trading_name',ascending = False)

In [60]:
gpB_df

,clue_small_area,trading_name
4,Melbourne (CBD),1738
1,Docklands,362
0,Carlton,352
9,Southbank,238
6,North Melbourne,132
10,West Melbourne (Residential),80
2,East Melbourne,76
5,Melbourne (Remainder),76
3,Kensington,64
8,Parkville,61
